## Linear Regression with a Real Dataset


In [1]:
# Import relevant modules
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

### Information about dataset
[California Housing Data Set Description
](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

In [2]:
# Import the dataset.
training_df = pd.read_csv(filepath_or_buffer="california_housing_train.csv")